In [1]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
import scipy.stats as stats

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

In [2]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived= '/Users/alina/Desktop/MIT/code/data/output/derived_data'

#os.listdir(data_root)

In [3]:

baseline_var = ['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site', 'days_baseline']
dtypes_baseline = { 'src_subject_id' : 'str',
                    'interview_date': 'str' , 
                    'interview_age' : 'int64' ,
                    'sex' : 'str', 
                    'site' : 'int64' ,
                    'days_baseline':  'int64',
                    'version_form': 'str'}

version_form = ['version_form']

qsts = ['snap', 'ssrs', 'masc', 'pc','wechsler']#, 'masc']##, 'wechsler'] #masc to many missing data 


In [4]:





snap_file = 'snap01.txt'
ssrs_file = 'ssrs01.txt'
masc_file = 'masc_p01.txt'
parent_child_file = 'pcrc01.txt'
wechsler_file = 'wiat_iiip201.txt'

treat_group_file = 'treatment_groups.csv'
treat_group = pd.read_csv(Path(data_derived, treat_group_file))



#outcome variablles 
snap_vars = ['snainatx', 'snahypax', 'snaoddx'] #inattention_mean, hyperactie mean
ssrs_vars = ['sspintx', 'ssptossx']# social skills mean, internalizing mean 
masc_vars = ['masc_masctotalt']
pc_vars = ['pcrcpax', 'pcrcprx'] # power assertion, personal closeness
wechsler_vars = ['w1readb','w2math','w3spell' ]
outcomes_dict  = {'snap' : snap_vars, 'ssrs' : ssrs_vars,  "masc": masc_vars, 'pc': pc_vars, 'wechsler': wechsler_vars}

interaction_predictors = ['days_baseline', 'site', 'trtname'] #time, site, treatment group

# mediator variables
# comorb_mediators  = ['cdorodd' , 'pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai' ,'psa'] #ODD/CD or anx excluding specific phobia 
# services_mediators =  ['demo61'] #reciept of public assistance 
# prev_med_mediators = ['hi_24'] #medication intake prior to study 

# #moderator variables 
# accept_moderator = ['d2dresp'] # initail acceptance of treatment 
# raters = ['Teacher', 'Parent']

treat_group = pd.read_csv(Path(data_derived, treat_group_file))


snap = prep.get_data(Path(data_root, snap_file), columns= [baseline_var, snap_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= None)
ssrs = prep.get_data(Path(data_root, ssrs_file), columns= [baseline_var, ssrs_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= None)
pc = prep.get_data(Path(data_root, parent_child_file), columns= [baseline_var, pc_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= None)
masc = prep.get_data(Path(data_root,masc_file), columns= [baseline_var, masc_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= None)
wechsler = prep.get_data(Path(data_root, wechsler_file), columns= [baseline_var, wechsler_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= None)
data_dict = dict(zip(qsts, [snap, ssrs, masc, pc, wechsler]))

Success
Success
Success
Success
Success


In [5]:
ssrs.loc[ssrs['version_form'].str.startswith('Teacher'), 'version_form'] = 'Teacher'
ssrs.loc[ssrs['version_form'].str.startswith('Parent'), 'version_form'] = 'Parent'
data_dict = dict(zip(qsts, [snap, ssrs, masc, pc, wechsler]))
for qst in qsts:
    data_dict[qst] = data_dict[qst][data_dict[qst]['trtname'] != 'L']
    
    # Optionally, remove 'L' from the category list if 'trtname' is categorical
    if data_dict[qst]['trtname'].dtype.name == 'category':
        data_dict[qst]['trtname'] = data_dict[qst]['trtname'].cat.remove_categories(['L'])

# Verify the changes
for qst in qsts[:2]:
    print(data_dict[qst].trtname.unique())

['C', 'M', 'A', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']
['M', 'A', 'C', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_33237/945923732.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_dict[qst]['trtname'] = data_dict[qst]['trtname'].cat.remove_categories(['L'])


## Include actual timepoints here

In [6]:
data_dict.keys()

dict_keys(['snap', 'ssrs', 'masc', 'pc', 'wechsler'])

In [7]:
timepoints = [46 , 168, 319, 500]

In [8]:
#snap = prep.get_data(Path(data_root, snap_file), columns= [baseline_var, snap_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= True)
#ssrs = prep.get_data(Path(data_root, ssrs_file), columns= [baseline_var, ssrs_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= True)
#pc = prep.get_data(Path(data_root, parent_child_file), columns= [baseline_var, pc_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= True)
#wechsler = prep.get_data(Path(data_root, wechsler_file), columns= [baseline_var, wechsler_vars], set_dtypes= True, version_form= False, split_timepoints= True)
#data_dict = dict(zip(qsts, [snap, ssrs, pc]))

In [9]:
#for key in data_dict.keys():
 #   for time, df in data_dict[key].items():
 #       data_dict[key][time] = df[(df['trtname'] != 'L')] #& (df['days_baseline']>=0)].copy() 

In [10]:
for key, data in data_dict.items():
    print(data.keys())

Index(['src_subject_id', 'interview_date', 'interview_age', 'sex', 'snainatx',
       'snahypax', 'snaoddx', 'days_baseline', 'site', 'version_form',
       'trtname'],
      dtype='object')
Index(['src_subject_id', 'interview_date', 'interview_age', 'sex',
       'days_baseline', 'site', 'version_form', 'ssptossx', 'sspintx',
       'trtname'],
      dtype='object')
Index(['src_subject_id', 'interview_date', 'interview_age', 'sex',
       'masc_masctotalt', 'days_baseline', 'site', 'trtname'],
      dtype='object')
Index(['src_subject_id', 'interview_date', 'interview_age', 'sex',
       'days_baseline', 'site', 'pcrcpax', 'pcrcprx', 'trtname'],
      dtype='object')
Index(['src_subject_id', 'interview_date', 'interview_age', 'sex',
       'days_baseline', 'site', 'w1readb', 'w2math', 'w3spell', 'trtname'],
      dtype='object')


In [11]:
data_dict["pc"]

,src_subject_id,interview_date,interview_age,sex,days_baseline,site,pcrcpax,pcrcprx,trtname
4,P1002,1997-06-02,120,M,0,1,3.83,3.2,C
5,P1002,1997-06-02,120,M,0,1,3.17,4.0,C
6,P1002,1997-09-18,124,M,108,1,3.25,3.8,C
7,P1002,1998-01-03,127,M,214,1,2.83,3.7,C
8,P1002,1998-01-13,127,M,224,1,2.50,3.7,C
...,...,...,...,...,...,...,...,...,...
8562,P1844,1995-09-17,77,M,147,4,2.75,3.6,A
8563,P1844,1996-05-04,84,M,377,4,1.83,3.2,A
8564,P1844,1996-05-04,84,M,377,4,2.42,3.1,A
8565,P1844,1997-03-11,95,M,687,4,1.75,4.3,A


In [155]:
results_table  = pd.read_csv(Path(data_derived, 'results_table_rr_and_new.csv'))
original_table = pd.read_csv(Path(data_derived, 'original_and_new_rr_14.csv'))

In [156]:
variables = original_table['variable']
variables = np.array(variables.dropna())
variables

array(['snainatx', 'snainatx', 'snainatx', 'snainatx', 'snainatx',
       'snainatx', 'snahypax', 'snahypax', 'snahypax', 'snahypax',
       'snahypax', 'snahypax', 'snaoddx', 'snaoddx', 'snaoddx', 'snaoddx',
       'snaoddx', 'snaoddx', 'sspintx', 'sspintx', 'sspintx', 'sspintx',
       'sspintx', 'sspintx', 'ssptossx', 'ssptossx', 'ssptossx',
       'ssptossx', 'ssptossx', 'ssptossx'], dtype=object)

In [157]:
outcomes_list = []
for out in outcomes_dict.values():
    outcomes_list.append(out)

In [158]:
outcomes_written = [['Inattention', 'Hyperactive', 'ODD'], 
    ['SSRS Internalizing', 'SSRS social skills'], 
    ['MASC'], 
    ['Power assertion', 'Personal'],
    ['eading'], ['Math'], ['Spelling']]
outcomes_written = np.concatenate(outcomes_written)
outcomes_list = np.concatenate(outcomes_list)


In [159]:

outcomes_dict_ = dict(zip(outcomes_list, outcomes_written))
outcomes_dict

{'snap': ['snainatx', 'snahypax', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'masc': ['masc_masctotalt'],
 'pc': ['pcrcpax', 'pcrcprx'],
 'wechsler': ['w1readb', 'w2math', 'w3spell']}

In [160]:
reversed_outcomes_dict = {v: k for k, v in outcomes_dict_.items()}

# Function to populate the 'variable' column based on the reversed dictionary
def populate_variable_column(row):
    for outcome_name, variable_key in reversed_outcomes_dict.items():
        # Check if the outcome name is contained in the 'Measure' column of the row
        if outcome_name in row['Measure']:
            return variable_key  # Return the corresponding key if a match is found
    return row['variable']  # Return the existing value if no match is found

# Apply the function to populate the 'variable' column
results_table['variable'] = results_table.apply(populate_variable_column, axis=1)

In [161]:
results_table

,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F-value,replicated P-value,variable
0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,7.114258,9.646144e-05,snainatx
1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,0.56,1.421438,1.288404e-01,snainatx
2,ADHD symptoms,Inattention,Teacher,Site,2.70,0.02,0.218636,9.546373e-01,snainatx
3,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,13.576264,8.953253e-09,snainatx
4,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,0.88,0.596591,8.797403e-01,snainatx
5,ADHD symptoms,Inattention,Parent,Site,3.00,0.02,0.330446,8.947980e-01,snainatx
6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,4.151560,6.125911e-03,snahypax
7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,0.49,1.084044,3.662135e-01,snahypax
8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,0.02,2.169635,5.513887e-02,snahypax
9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,13.364109,1.214601e-08,snahypax


In [162]:

# results_table = results_table.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

# results_table_cols_order = results_table.columns
# index_m_r = results_table.columns.get_loc('Measure and Rater')
# index_m_r

# results_table.loc[results_table['Measure and Rater'] == 'SSRS Teacher', 'Measure and Rater'] = 'SSRS - Teacher'
# results_table.loc[results_table['Measure and Rater'] == 'SSRS Parent', 'Measure and Rater'] = 'SSRS - Parent'

# results_table[['Measure', 'Rater']] = results_table['Measure and Rater'].str.split(' - ', expand=True)
# results_table = results_table.drop(columns=['Measure and Rater'])

# results_table.insert(index_m_r, 'Measure', results_table.pop('Measure'))
# results_table.insert(index_m_r + 1, 'Rater', results_table.pop('Rater'))
# results_table

### Contrasts 1 : \
- (M+C) vs (P + CC)
- M vs C
- B vs CC

In [20]:
#data_dict['snap']['b']['trtname'].value_counts()

In [21]:
def get_contrast_matrix(paper = 'original', new_order = None):
    if paper == 'original':
        
        contrast_matrix = np.array([ # add 0 column for intercept generated from RR, ignored bc coefficients for contrasts = 0
            [ 1, -1, 0, 0],  # M vs P
            [ -1, 0, 1, 0],  # C vs M
            [ 0, -1, 1, 0],  # C vs P
            [ -1, 0, 0, 1],  # A vs M
            [ 0, -1, 0, 1],  # A vs P
            [ 0, 0, -1, 1]   # A vs C
        ])

    elif paper == 'molina':
        contrast_matrix = np.array([[  1, -1	,1,-1],   # (M,C) vs (B,CC)
                                    [  1, 0,	-1,	0],   # (M vs C)
                                    [  0, 1	,0,-1]])  # (B vs CC)
  
    contrast_m = pd.DataFrame(contrast_matrix, columns=['M', 'P', 'C', 'A'])
    if new_order is not None:
        contrast_m = contrast_m[new_order]
    return contrast_m
        

In [22]:
def find_first_index(df, column, value, condition = None):
    if condition is not None:
        condition_column, condition_value = condition.split(' == ')
        return df[(df[column] == value) & (df[condition_column] == condition_value)].index[0]
    else:
        return (df[column] == value).idxmax()

In [23]:
outcomes_dict

{'snap': ['snainatx', 'snahypax', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'masc': ['masc_masctotalt'],
 'pc': ['pcrcpax', 'pcrcprx'],
 'wechsler': ['w1readb', 'w2math', 'w3spell']}

In [24]:
data_dict.keys()

dict_keys(['snap', 'ssrs', 'masc', 'pc', 'wechsler'])

In [25]:
qst = 'ssrs'
outcome = outcomes_dict[qst][0]
predictor = 'trtname'
data = data_dict[qst]
rater = 'Teacher'

In [26]:
n_obs = data.shape[0]
alpha = 0.05
bonferroni = False
type_contrast = 'original'
formula =' ~ '.join((outcome, predictor))
#data = data[data['version_form'] == 'Teacher']

In [27]:
data.trtname.value_counts()

trtname
C    2228
P    2156
A    2089
M    2032
Name: count, dtype: int64

In [28]:
def perform_contrasts(data, outcome, predictor, type_contrast, alpha, bonferroni):
    
    predictor = 'C(trtname, Treatment(reference="A"))'
    if bonferroni : 
        alpha = alpha / n_obs 

    formula =' ~ '.join((outcome, predictor)) #+ " * days_baseline"
    print(formula)
    # change when get access to CC data, careful with interpretations !
    contrasts_matrix = get_contrast_matrix(type_contrast)
    #print(contrasts_matrix)
    #print(data[outcome].shape, data[outcome].dropna().shape)

    model = smf.mixedlm(formula, data = data, groups='src_subject_id')

    result = model.fit()
    print(result.model.exog_names)
    print("OLD ORDER:",contrasts_matrix.columns)
    new_order = ['A' if 'Intercept' in item else item.split('[')[1][2] for item in result.model.exog_names]
    print("NEW ORDER:" , new_order)
    contrasts_matrix = contrasts_matrix[new_order]



    contrasts = result.t_test(contrasts_matrix)

    coef = contrasts.effect
    stderr = contrasts.sd
    z_scores = contrasts.tvalue
    p_values = contrasts.pvalue.round(4)
    conf_int = contrasts.conf_int()

    significant =  p_values < alpha

    comp_results_str = []

    for i, val in enumerate(significant): 

        if val and coef[i] > 0 :
            results_sign = ' < ' # this would be inverted if we were comparing the means, but we want to know which treatment DECREASE the outcome variable
        elif val and coef[i] < 0 :
            results_sign = ' > '
        else: 
            results_sign = ' ~ '
        comp = np.array(contrasts_matrix.columns[contrasts_matrix.iloc[i] != 0])
        comp_results_str.append(results_sign.join(comp))
    
    return {str(outcome): comp_results_str, 'p_val' + str(outcome): p_values}


In [29]:
data  = data_dict['ssrs']#['14']
data.trtname.value_counts(0)

trtname
C    2228
P    2156
A    2089
M    2032
Name: count, dtype: int64

In [30]:
all_outcomes = np.concatenate(list(outcomes_dict.values()))
all_outcomes

array(['snainatx', 'snahypax', 'snaoddx', 'sspintx', 'ssptossx',
       'masc_masctotalt', 'pcrcpax', 'pcrcprx', 'w1readb', 'w2math',
       'w3spell'], dtype='<U15')

In [31]:
data_dict

{'snap':       src_subject_id interview_date  interview_age sex  snainatx  snahypax  \
 6              P1002     1997-05-02            155   M      2.00      1.17   
 7              P1002     1997-06-03            120   M      1.00      0.50   
 8              P1002     1997-06-03            120   M      1.11      1.00   
 9              P1002     1997-09-19            124   M      0.33      0.33   
 10             P1002     1998-01-03            127   M      0.44      0.17   
 ...              ...            ...            ...  ..       ...       ...   
 14394          P1844     1995-04-24             72   M      3.00      2.33   
 14395          P1844     1995-10-04             77   M      2.11      1.00   
 14396          P1844     1996-04-05             84   M      3.00      1.50   
 14397          P1844     1997-03-26             95   M      1.89      1.17   
 14398          P1844     1998-03-21            107   M      1.78      1.33   
 
        snaoddx  days_baseline site versio

In [32]:
data.trtname.value_counts()

trtname
C    2228
P    2156
A    2089
M    2032
Name: count, dtype: int64

In [33]:
data[data['version_form'] == 'Teacher'].trtname.value_counts()

trtname
C    1084
P    1058
A    1010
M     989
Name: count, dtype: int64

In [34]:
outcomes_dict

{'snap': ['snainatx', 'snahypax', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'masc': ['masc_masctotalt'],
 'pc': ['pcrcpax', 'pcrcprx'],
 'wechsler': ['w1readb', 'w2math', 'w3spell']}

In [35]:
data_dict.keys()

dict_keys(['snap', 'ssrs', 'masc', 'pc', 'wechsler'])

In [36]:
raters = ['Teacher', 'Parent']

In [37]:
contrast_dict , p_vals_dict ={}, {}
contrast_list = []


for qst in data_dict.keys():
    data = data_dict[qst][data_dict[qst]['days_baseline'] < timepoints[1]]
    print(qst)

    
    #data= data[data['trtname'] != 'L']
    print(data.trtname.value_counts())

    outcomes = outcomes_dict[qst]
    print(outcomes)
    version_form = raters if qst == 'snap' or qst == 'ssrs'  else None
    contrasts =[]

    print(qst, data.shape, outcomes, predictor, version_form, alpha)
    contrast = []
    if version_form is not None:
        for rater in raters:
            for out in outcomes: 

                print(out, rater)
                contrast.append(perform_contrasts(data[data['version_form'] == rater], out, predictor, type_contrast='original',  alpha= alpha, bonferroni=False))
    else :

        for out in outcomes:
            print(out)
            contrast.append(perform_contrasts(data, out, predictor, type_contrast='original',  alpha= alpha, bonferroni=False) )

            

    contrast_dict[qst] = contrast
    contrast_list.append(contrast)
       # print(version_form)
    

snap
trtname
C    905
P    893
A    875
M    871
Name: count, dtype: int64
['snainatx', 'snahypax', 'snaoddx']
snap (3544, 11) ['snainatx', 'snahypax', 'snaoddx'] trtname ['Teacher', 'Parent'] 0.05
snainatx Teacher
snainatx ~ C(trtname, Treatment(reference="A"))
['Intercept', 'C(trtname, Treatment(reference="A"))[T.C]', 'C(trtname, Treatment(reference="A"))[T.M]', 'C(trtname, Treatment(reference="A"))[T.P]']
OLD ORDER: Index(['M', 'P', 'C', 'A'], dtype='object')
NEW ORDER: ['A', 'C', 'M', 'P']
snahypax Teacher
snahypax ~ C(trtname, Treatment(reference="A"))
['Intercept', 'C(trtname, Treatment(reference="A"))[T.C]', 'C(trtname, Treatment(reference="A"))[T.M]', 'C(trtname, Treatment(reference="A"))[T.P]']
OLD ORDER: Index(['M', 'P', 'C', 'A'], dtype='object')
NEW ORDER: ['A', 'C', 'M', 'P']
snaoddx Teacher
snaoddx ~ C(trtname, Treatment(reference="A"))
['Intercept', 'C(trtname, Treatment(reference="A"))[T.C]', 'C(trtname, Treatment(reference="A"))[T.M]', 'C(trtname, Treatment(reference="

/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


['Intercept', 'C(trtname, Treatment(reference="A"))[T.C]', 'C(trtname, Treatment(reference="A"))[T.M]', 'C(trtname, Treatment(reference="A"))[T.P]']
OLD ORDER: Index(['M', 'P', 'C', 'A'], dtype='object')
NEW ORDER: ['A', 'C', 'M', 'P']
w2math
w2math ~ C(trtname, Treatment(reference="A"))


/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


['Intercept', 'C(trtname, Treatment(reference="A"))[T.C]', 'C(trtname, Treatment(reference="A"))[T.M]', 'C(trtname, Treatment(reference="A"))[T.P]']
OLD ORDER: Index(['M', 'P', 'C', 'A'], dtype='object')
NEW ORDER: ['A', 'C', 'M', 'P']
w3spell
w3spell ~ C(trtname, Treatment(reference="A"))
['Intercept', 'C(trtname, Treatment(reference="A"))[T.C]', 'C(trtname, Treatment(reference="A"))[T.M]', 'C(trtname, Treatment(reference="A"))[T.P]']
OLD ORDER: Index(['M', 'P', 'C', 'A'], dtype='object')
NEW ORDER: ['A', 'C', 'M', 'P']


In [38]:
contrast_dict


{'snap': [{'snainatx': ['M > P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnainatx': array([0.0478, 0.0432, 0.0001, 0.    , 0.    , 0.    ])},
  {'snahypax': ['M ~ P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnahypax': array([6.65e-01, 2.60e-03, 6.00e-04, 0.00e+00, 0.00e+00, 0.00e+00])},
  {'snaoddx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnaoddx': array([0.3011, 0.2581, 0.03  , 0.    , 0.    , 0.    ])},
  {'snainatx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnainatx': array([0.3197, 0.1975, 0.0217, 0.    , 0.    , 0.    ])},
  {'snahypax': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnahypax': array([0.0993, 0.4426, 0.0151, 0.    , 0.    , 0.    ])},
  {'snaoddx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
   'p_valsnaoddx': array([0.3868, 0.0579, 0.303 , 0.    , 0.    , 0.    ])}],
 'ssrs': [{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
   'p_valss

In [39]:
for i in range(len(qsts)):
    print(contrast_list[i])

[{'snainatx': ['M > P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnainatx': array([0.0478, 0.0432, 0.0001, 0.    , 0.    , 0.    ])}, {'snahypax': ['M ~ P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnahypax': array([6.65e-01, 2.60e-03, 6.00e-04, 0.00e+00, 0.00e+00, 0.00e+00])}, {'snaoddx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnaoddx': array([0.3011, 0.2581, 0.03  , 0.    , 0.    , 0.    ])}, {'snainatx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnainatx': array([0.3197, 0.1975, 0.0217, 0.    , 0.    , 0.    ])}, {'snahypax': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnahypax': array([0.0993, 0.4426, 0.0151, 0.    , 0.    , 0.    ])}, {'snaoddx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'], 'p_valsnaoddx': array([0.3868, 0.0579, 0.303 , 0.    , 0.    , 0.    ])}]
[{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'], 'p_valsspintx': array([0.1324, 0.7879, 0.0746, 0.    , 0. 

In [40]:

# qst , #0,1 rater, # 0,1 qst_var , # var or pval_val

In [163]:
all_outcomes_list = [['snainatx', 'snahypax', 'snaoddx'],['sspintx', 'ssptossx'],['pcrcpax', 'pcrcprx']]#, ['w1readb', 'w2math', 'w3spell']]
all_outcomes_list

[['snainatx', 'snahypax', 'snaoddx'],
 ['sspintx', 'ssptossx'],
 ['pcrcpax', 'pcrcprx']]

In [164]:
for qst in qsts:
    print(outcomes_dict[qst])

['snainatx', 'snahypax', 'snaoddx']
['sspintx', 'ssptossx']
['masc_masctotalt']
['pcrcpax', 'pcrcprx']
['w1readb', 'w2math', 'w3spell']


In [165]:
contrast_list[0]

[{'snainatx': ['M > P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'],
  'p_valsnainatx': array([0.0478, 0.0432, 0.0001, 0.    , 0.    , 0.    ])},
 {'snahypax': ['M ~ P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'],
  'p_valsnahypax': array([6.65e-01, 2.60e-03, 6.00e-04, 0.00e+00, 0.00e+00, 0.00e+00])},
 {'snaoddx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
  'p_valsnaoddx': array([0.3011, 0.2581, 0.03  , 0.    , 0.    , 0.    ])},
 {'snainatx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
  'p_valsnainatx': array([0.3197, 0.1975, 0.0217, 0.    , 0.    , 0.    ])},
 {'snahypax': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
  'p_valsnahypax': array([0.0993, 0.4426, 0.0151, 0.    , 0.    , 0.    ])},
 {'snaoddx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
  'p_valsnaoddx': array([0.3868, 0.0579, 0.303 , 0.    , 0.    , 0.    ])}]

In [166]:
qsts

['snap', 'ssrs', 'masc', 'pc', 'wechsler']

In [167]:
contrast_dict

{'snap': [{'snainatx': ['M > P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnainatx': array([0.0478, 0.0432, 0.0001, 0.    , 0.    , 0.    ])},
  {'snahypax': ['M ~ P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnahypax': array([6.65e-01, 2.60e-03, 6.00e-04, 0.00e+00, 0.00e+00, 0.00e+00])},
  {'snaoddx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnaoddx': array([0.3011, 0.2581, 0.03  , 0.    , 0.    , 0.    ])},
  {'snainatx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnainatx': array([0.3197, 0.1975, 0.0217, 0.    , 0.    , 0.    ])},
  {'snahypax': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnahypax': array([0.0993, 0.4426, 0.0151, 0.    , 0.    , 0.    ])},
  {'snaoddx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
   'p_valsnaoddx': array([0.3868, 0.0579, 0.303 , 0.    , 0.    , 0.    ])}],
 'ssrs': [{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
   'p_valss

In [168]:
results_table.columns

Index(['Outcome Domain', 'Measure', 'Rater', 'Test', 'F_value', 'P_value',
       'replicated F-value', 'replicated P-value', 'variable'],
      dtype='object')

In [169]:
contrast_dict

{'snap': [{'snainatx': ['M > P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnainatx': array([0.0478, 0.0432, 0.0001, 0.    , 0.    , 0.    ])},
  {'snahypax': ['M ~ P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnahypax': array([6.65e-01, 2.60e-03, 6.00e-04, 0.00e+00, 0.00e+00, 0.00e+00])},
  {'snaoddx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnaoddx': array([0.3011, 0.2581, 0.03  , 0.    , 0.    , 0.    ])},
  {'snainatx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnainatx': array([0.3197, 0.1975, 0.0217, 0.    , 0.    , 0.    ])},
  {'snahypax': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'],
   'p_valsnahypax': array([0.0993, 0.4426, 0.0151, 0.    , 0.    , 0.    ])},
  {'snaoddx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
   'p_valsnaoddx': array([0.3868, 0.0579, 0.303 , 0.    , 0.    , 0.    ])}],
 'ssrs': [{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
   'p_valss

In [170]:
outcomes_dict

{'snap': ['snainatx', 'snahypax', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'masc': ['masc_masctotalt'],
 'pc': ['pcrcpax', 'pcrcprx'],
 'wechsler': ['w1readb', 'w2math', 'w3spell']}

In [171]:
contrast_list[1]

[{'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
  'p_valsspintx': array([0.1324, 0.7879, 0.0746, 0.    , 0.    , 0.    ])},
 {'ssptossx': ['M ~ P', 'C ~ M', 'C < P', 'A < M', 'A < P', 'A < C'],
  'p_valssptossx': array([0.0713, 0.1862, 0.0017, 0.    , 0.    , 0.    ])},
 {'sspintx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
  'p_valsspintx': array([0.9002, 0.8577, 0.7592, 0.    , 0.    , 0.    ])},
 {'ssptossx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'],
  'p_valssptossx': array([0.4208, 0.0516, 0.2524, 0.    , 0.    , 0.    ])}]

In [172]:
snap_c = contrast_list[0]
print(snap_c)
snap_c_t = contrast_list[0][0]
snap_c_p = contrast_list[0][1]
print(snap_c_p, snap_c_t)
# for i, var in enumerate(snap_vars):
#     print(snap_c_t[i][var])
#     print(snap_c_t[i]['p_val' + var])

[{'snainatx': ['M > P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnainatx': array([0.0478, 0.0432, 0.0001, 0.    , 0.    , 0.    ])}, {'snahypax': ['M ~ P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnahypax': array([6.65e-01, 2.60e-03, 6.00e-04, 0.00e+00, 0.00e+00, 0.00e+00])}, {'snaoddx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnaoddx': array([0.3011, 0.2581, 0.03  , 0.    , 0.    , 0.    ])}, {'snainatx': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnainatx': array([0.3197, 0.1975, 0.0217, 0.    , 0.    , 0.    ])}, {'snahypax': ['M ~ P', 'C ~ M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnahypax': array([0.0993, 0.4426, 0.0151, 0.    , 0.    , 0.    ])}, {'snaoddx': ['M ~ P', 'C ~ M', 'C ~ P', 'A < M', 'A < P', 'A < C'], 'p_valsnaoddx': array([0.3868, 0.0579, 0.303 , 0.    , 0.    , 0.    ])}]
{'snahypax': ['M ~ P', 'C > M', 'C > P', 'A < M', 'A < P', 'A < C'], 'p_valsnahypax': array([6.65e-01, 2.60e-03, 6.00e-04, 0.00

In [173]:
all_outcomes_list

[['snainatx', 'snahypax', 'snaoddx'],
 ['sspintx', 'ssptossx'],
 ['pcrcpax', 'pcrcprx']]

In [174]:
results_table

,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F-value,replicated P-value,variable
0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,7.114258,9.646144e-05,snainatx
1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,0.56,1.421438,1.288404e-01,snainatx
2,ADHD symptoms,Inattention,Teacher,Site,2.70,0.02,0.218636,9.546373e-01,snainatx
3,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,13.576264,8.953253e-09,snainatx
4,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,0.88,0.596591,8.797403e-01,snainatx
5,ADHD symptoms,Inattention,Parent,Site,3.00,0.02,0.330446,8.947980e-01,snainatx
6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,4.151560,6.125911e-03,snahypax
7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,0.49,1.084044,3.662135e-01,snahypax
8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,0.02,2.169635,5.513887e-02,snahypax
9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,13.364109,1.214601e-08,snahypax


In [175]:
results_table.loc[results_table['Measure'] == 'Power assertion', 'variable'] = pc_vars[0]
results_table.loc[results_table['Measure'] == 'Personal closeness'  ,  'variable'] = pc_vars[1]
results_table['M vs B'] = pd.NA
results_table['M vs B p_val'] = pd.NA
results_table['C vs (M,B)'] = pd.NA
results_table['C vs (M,B) p_val'] = pd.NA
results_table['A vs (M,B,C)'] = pd.NA
results_table['A vs (M,B,C) p_val'] = pd.NA
results_table


,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F-value,replicated P-value,variable,M vs B,M vs B p_val,"C vs (M,B)","C vs (M,B) p_val","A vs (M,B,C)","A vs (M,B,C) p_val"
0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,7.114258,9.646144e-05,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,0.56,1.421438,1.288404e-01,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,ADHD symptoms,Inattention,Teacher,Site,2.70,0.02,0.218636,9.546373e-01,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,13.576264,8.953253e-09,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,0.88,0.596591,8.797403e-01,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,ADHD symptoms,Inattention,Parent,Site,3.00,0.02,0.330446,8.947980e-01,snainatx,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,4.151560,6.125911e-03,snahypax,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,0.49,1.084044,3.662135e-01,snahypax,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,0.02,2.169635,5.513887e-02,snahypax,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,13.364109,1.214601e-08,snahypax,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [176]:
outcomes_dict

{'snap': ['snainatx', 'snahypax', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'masc': ['masc_masctotalt'],
 'pc': ['pcrcpax', 'pcrcprx'],
 'wechsler': ['w1readb', 'w2math', 'w3spell']}

In [177]:
raters = ["Teacher", "Parent"] 

## CONTINUE HERE

In [178]:

for i,  qst in enumerate(['snap', 'ssrs', 'pc', 'wechsler']):
    qst_c = contrast_dict[qst]
    outcomes = outcomes_dict[qst]

    #print(qst_c)
    if qst == 'snap' or qst == 'ssrs' :
        for rater in raters : 
            for i, var in enumerate(outcomes):
                

                result = qst_c[i][var]
                result_p_val = qst_c[i]["p_val" + var]
                #print(qst, var, result , result_p_val)
                M_P = result[0]
                M_P_p_val = result_p_val[0]
                
                C_MP = result[1:3]
                C_MP_p_val = result_p_val[1:3]
                
                L_CMP = result[3:]
                L_CMP_p_val = result_p_val[3:]
                print(M_P, C_MP, L_CMP)
                print(M_P_p_val, C_MP_p_val, L_CMP_p_val)
                first_index = results_table.index[results_table['variable'] == var].tolist()[0] if rater == 'Teacher' else results_table.index[results_table['variable'] == var].tolist()[0] +3
                
                print(var, rater, first_index)
                results_table.loc[first_index, 'M vs B'] = M_P
                results_table.loc[first_index:first_index+1, 'C vs (M,B)'] = C_MP
                results_table.loc[first_index:first_index+2, 'A vs (M,B,C)'] = L_CMP
                
                results_table.loc[first_index, 'M vs B p_val'] = M_P_p_val
                results_table.loc[first_index:first_index+1, 'C vs (M,B) p_val'] = C_MP_p_val
                results_table.loc[first_index:first_index+2, 'A vs (M,B,C) p_val'] = L_CMP_p_val
    else: 
        for i, var in enumerate(outcomes):
            result = qst_c[i][var]
            result_p_val = qst_c[i]["p_val" + var]
            print(qst, var, result, result_p_val )
            M_P = result[0]
            M_P_p_val = result_p_val[0]
            
            C_MP = result[1:3]
            C_MP_p_val = result_p_val[1:3]
            
            L_CMP = result[3:]
            L_CMP_p_val = result_p_val[3:]
            print(len(M_P), len(C_MP), len(L_CMP))
            print(M_P_p_val, len(C_MP_p_val), len(L_CMP_p_val))
            first_index = results_table.index[results_table['variable'] == var].tolist()[0]
            
            print(var, first_index)
            results_table.loc[first_index, 'M vs B'] = M_P
            results_table.loc[first_index:first_index+1, 'C vs (M,B)'] = C_MP
            results_table.loc[first_index:first_index+2, 'A vs (M,B,C)'] = L_CMP
            
            results_table.loc[first_index, 'M vs B p_val'] = M_P_p_val
            results_table.loc[first_index:first_index+1, 'C vs (M,B) p_val'] = C_MP_p_val
            results_table.loc[first_index:first_index+2, 'A vs (M,B,C) p_val'] = L_CMP_p_val
       
        
        #result_pval = qst_c[i]["p_val" + var]
    
        
        
        #result_pval = qst_c[i]["pval"+ var]
        
        

    


M > P ['C > M', 'C > P'] ['A < M', 'A < P', 'A < C']
0.0478 [0.0432 0.0001] [0. 0. 0.]
snainatx Teacher 0
M ~ P ['C > M', 'C > P'] ['A < M', 'A < P', 'A < C']
0.665 [0.0026 0.0006] [0. 0. 0.]
snahypax Teacher 6
M ~ P ['C ~ M', 'C > P'] ['A < M', 'A < P', 'A < C']
0.3011 [0.2581 0.03  ] [0. 0. 0.]
snaoddx Teacher 12
M > P ['C > M', 'C > P'] ['A < M', 'A < P', 'A < C']
0.0478 [0.0432 0.0001] [0. 0. 0.]
snainatx Parent 3
M ~ P ['C > M', 'C > P'] ['A < M', 'A < P', 'A < C']
0.665 [0.0026 0.0006] [0. 0. 0.]
snahypax Parent 9
M ~ P ['C ~ M', 'C > P'] ['A < M', 'A < P', 'A < C']
0.3011 [0.2581 0.03  ] [0. 0. 0.]
snaoddx Parent 15
M ~ P ['C ~ M', 'C ~ P'] ['A < M', 'A < P', 'A < C']
0.1324 [0.7879 0.0746] [0. 0. 0.]
sspintx Teacher 18
M ~ P ['C ~ M', 'C < P'] ['A < M', 'A < P', 'A < C']
0.0713 [0.1862 0.0017] [0. 0. 0.]
ssptossx Teacher 27
M ~ P ['C ~ M', 'C ~ P'] ['A < M', 'A < P', 'A < C']
0.1324 [0.7879 0.0746] [0. 0. 0.]
sspintx Parent 21
M ~ P ['C ~ M', 'C < P'] ['A < M', 'A < P', 'A < C'

In [179]:
comparison_list = [
    'M > P', 
    'M > P', 
    'M ~ P', 
    
    'M > P', 
    'M ~ P',
    'M ~ P', 
    '...',
    
    'M ~ P',
    'M ~ P', 
    'M ~ P', 
    'M ~ P', 
    
    '...',
    '...', # wechsler 
    '...',
    '...'
]

comparison_list_fill = [
    'M > P', 
    'M > P', 
    'M ~ P', 
    
    'M > P', 
    'M ~ P',
    'M ~ P', 
    
    '...',
    'M ~ P',
    'M ~ P', 
    'M ~ P', 
    'M ~ P', 
    '...',
    '...', # wechsler 
    '...',
    '...'
]

comparison_list_c = [
    'C > M', 'C > P',
    
    'C > M', 'C > P',
    
    'C ~ M', 'C ~ P',
    
    'C ~ M', 'C > P',
    
    'C ~ M', 'C ~ P',
    
    'C ~ M', 'C > P',
    
    '...', '...',

    
    'C ~ M', 'C > P',
    
    'C ~ M', 'C ~ P',
    'C ~ M', 'C ~ P',
    'C ~ M', 'C ~ P',
    
    '...', '...',
    
    '...', '...', #wechsler
    '...', '...',
    '...', '...'

]
print(len(comparison_list_c))



comparison_list_a = [
    'A < M', 'A ~ P', 'A < C',
    
    'A < M', 'A ~ P', 'A < C',
    
    'A < M', 'A ~ P', 'A < C',
    
    'A < M', 'A ~ P', 'A < C',
    
    'A < M', 'A ~ P', 'A < C',
    
    
    
    'A ~ M', 'A ~ P', 'A < C',
    
    
    '...', '...', '...',

    
    'A ~ M', 'A ~ P', 'A < C',
    
    
    'A ~ M', 'A ~ P', 'A < C',
    
    
    
    'A < M', 'A ~ P', 'A < C',
    
    'A ~ M', 'A ~ P', 'A < C',
    
    
    
    '...', '...', '...', 
    '...', '...', '...',
    '...', '...', '...',
    '...', '...', '...'

]

30


In [180]:
results_table = results_table.rename(columns={
    'M vs B': 'replicated M vs B',
    'C vs (M,B)': 'replicated C vs (M,B)',
    'A vs (M,B,C)': 'replicated A vs (M,B,C)'
})

# Insert new columns for the original data
results_table.insert(results_table.columns.get_loc('replicated M vs B'), 'original M vs B', None)
results_table.insert(results_table.columns.get_loc('replicated C vs (M,B)'), 'original C vs (M,B)', None)
results_table.insert(results_table.columns.get_loc('replicated A vs (M,B,C)'), 'original A vs (M,B,C)', None)

In [181]:
print(len)

<built-in function len>


In [182]:
indexes_MB = results_table[results_table['replicated M vs B'].notna()].index
print(len(indexes_MB))
results_table['original M vs B'][indexes_MB] = comparison_list_fill



15


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_33237/3865342766.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  results_table['original M vs B'][indexes_MB] = comparison_list_fill
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000g

In [183]:
indexes_C= results_table[results_table['replicated C vs (M,B)'].notna()].index
print(len(indexes_C))
results_table['original C vs (M,B)'][indexes_C] = comparison_list_c

30


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_33237/1610208508.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  results_table['original C vs (M,B)'][indexes_C] = comparison_list_c
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000g

In [184]:
indexes_A= results_table[results_table['replicated A vs (M,B,C)'].notna()].index
print(len(indexes_A))


45


In [185]:

results_table['original A vs (M,B,C)'][indexes_A] = comparison_list_a

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_33237/1482802709.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  results_table['original A vs (M,B,C)'][indexes_A] = comparison_list_a
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh000

In [186]:
len(comparison_list_a)

45

In [187]:
results_table

,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F-value,replicated P-value,variable,original M vs B,replicated M vs B,M vs B p_val,"original C vs (M,B)","replicated C vs (M,B)","C vs (M,B) p_val","original A vs (M,B,C)","replicated A vs (M,B,C)","A vs (M,B,C) p_val"
0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,7.114258,9.646144e-05,snainatx,M > P,M > P,0.0478,C > M,C > M,0.0432,A < M,A < M,0.0
1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,0.56,1.421438,1.288404e-01,snainatx,None,<NA>,<NA>,C > P,C > P,0.0001,A ~ P,A < P,0.0
2,ADHD symptoms,Inattention,Teacher,Site,2.70,0.02,0.218636,9.546373e-01,snainatx,None,<NA>,<NA>,None,<NA>,<NA>,A < C,A < C,0.0
3,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,13.576264,8.953253e-09,snainatx,M > P,M > P,0.0478,C > M,C > M,0.0432,A < M,A < M,0.0
4,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,0.88,0.596591,8.797403e-01,snainatx,None,<NA>,<NA>,C > P,C > P,0.0001,A ~ P,A < P,0.0
5,ADHD symptoms,Inattention,Parent,Site,3.00,0.02,0.330446,8.947980e-01,snainatx,None,<NA>,<NA>,None,<NA>,<NA>,A < C,A < C,0.0
6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,4.151560,6.125911e-03,snahypax,M ~ P,M ~ P,0.665,C ~ M,C > M,0.0026,A < M,A < M,0.0
7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,0.49,1.084044,3.662135e-01,snahypax,None,<NA>,<NA>,C ~ P,C > P,0.0006,A ~ P,A < P,0.0
8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,0.02,2.169635,5.513887e-02,snahypax,None,<NA>,<NA>,None,<NA>,<NA>,A < C,A < C,0.0
9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,13.364109,1.214601e-08,snahypax,M > P,M ~ P,0.665,C ~ M,C > M,0.0026,A < M,A < M,0.0


In [188]:
#results_table = results_table.drop(columns=['F_value', 'P_value','replicated F-value', 'replicated P-value', 'Test', 'variable'])
      
results_table = results_table[results_table['Measure'] != 'MASC Child']

all_except_MB = results_table.index.difference(indexes_MB)

# Set specified columns to pd.NA for all indices except those in indexes_MB
results_table.loc[all_except_MB, ['Outcome Domain', 'Measure', 'Rater']] = pd.NA
      # 'replicated F-value', 'replicated P-value'])

In [191]:
non_nan_indices_M_vs_B = results_table[results_table['original M vs B'].notna()].index
non_nan_indices_C_vs_MB = results_table[results_table['original C vs (M,B)'].notna()].index

# Find indices with NaN values in 'original M vs B' and 'original C vs (M,B)'
nan_indices_M_vs_B = results_table.index.difference(non_nan_indices_M_vs_B)
nan_indices_C_vs_MB = results_table.index.difference(non_nan_indices_C_vs_MB)

# Set NaN values in these indices to pd.NA
results_table.loc[nan_indices_M_vs_B, 'original M vs B'] = pd.NA
results_table.loc[nan_indices_C_vs_MB, 'original C vs (M,B)'] = pd.NA

In [192]:
def highlight_comparison(cell, comparison):
    if pd.isna(cell):
        return ''  # No color for NaN values
    elif comparison == '...':
        return 'background-color: gray'
    elif cell == comparison:
        return 'background-color: green'
    else:
        return 'background-color: red'

def highlight_p_values_row(row):
    # Check if both 'P_value' and 'replicated P-value' are below 0.05 or contain '<.001'
    p_value = row['P_value']
    replicated_p_value = row['replicated P-value']
    
    # Determine if each cell should be highlighted blue or green
    if ((isinstance(p_value, str) and p_value == '<.001') or (isinstance(p_value, float) and p_value < 0.05)) and \
       ((isinstance(replicated_p_value, str) and replicated_p_value == '<.001') or (isinstance(replicated_p_value, float) and replicated_p_value < 0.05)):
        return ['background-color: green'] * len(row)  # Entire row green if both conditions are met
    else:
        # Apply blue background to individual cells as needed
        return [
            'background-color: blue' if (isinstance(val, str) and val == '<.001') or (isinstance(val, float) and val < 0.05) else ''
            for val in row
        ]

def highlight_simple(val):
    if (isinstance(val, str) and val == '<.001') or (isinstance(val, float) and val < 0.05):
        return 'background-color: blue'
    return ''  # No background color if conditions are not met

# Initialize lists to store styles for each column
styles_m_vs_b = []
styles_c_vs_mb = []
styles_a_vs_mbc = []

# Track indexes for each comparison list
comparison_index_m = 0
comparison_index_c = 0
comparison_index_a = 0

# Loop through each cell in the columns 'M vs B', 'C vs (M,B)', and 'A vs (M,B,C)'
for cell_m, cell_c, cell_a in zip(results_table['replicated M vs B'], results_table['replicated C vs (M,B)'], results_table['replicated A vs (M,B,C)']):
    # For 'M vs B'
    if not pd.isna(cell_m):
        comparison_m = comparison_list[comparison_index_m]
        styles_m_vs_b.append(highlight_comparison(cell_m, comparison_m))
        comparison_index_m += 1
    else:
        styles_m_vs_b.append('')
    
    # For 'C vs (M,B)'
    if not pd.isna(cell_c):
        comparison_c = comparison_list_c[comparison_index_c]
        styles_c_vs_mb.append(highlight_comparison(cell_c, comparison_c))
        comparison_index_c += 1
    else:
        styles_c_vs_mb.append('')

    # For 'A vs (M,B,C)'
    if not pd.isna(cell_a):
        comparison_a = comparison_list_a[comparison_index_a]
        styles_a_vs_mbc.append(highlight_comparison(cell_a, comparison_a))
        comparison_index_a += 1
    else:
        styles_a_vs_mbc.append('')

# Apply all styling functions to the DataFrame
styled_results_table = (
    results_table.style
.applymap(highlight_simple, subset=['M vs B p_val', 'C vs (M,B) p_val', 'A vs (M,B,C) p_val'])
    .apply(lambda _: styles_m_vs_b, subset=['replicated M vs B'])
    .apply(lambda _: styles_c_vs_mb, subset=['replicated C vs (M,B)'])
    .apply(lambda _: styles_a_vs_mbc, subset=['replicated A vs (M,B,C)'])
)

# Display the styled table with all applied highlights
styled_results_table

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_33237/3739465185.py:70: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  results_table.style


,Outcome Domain,Measure,Rater,original M vs B,replicated M vs B,M vs B p_val,"original C vs (M,B)","replicated C vs (M,B)","C vs (M,B) p_val","original A vs (M,B,C)","replicated A vs (M,B,C)","A vs (M,B,C) p_val"
0,ADHD symptoms,Inattention,Teacher,M > P,M > P,0.047800,C > M,C > M,0.043200,A < M,A < M,0.000000
1,,,,,,,C > P,C > P,0.000100,A ~ P,A < P,0.000000
2,,,,,,,,,,A < C,A < C,0.000000
3,ADHD symptoms,Inattention,Parent,M > P,M > P,0.047800,C > M,C > M,0.043200,A < M,A < M,0.000000
4,,,,,,,C > P,C > P,0.000100,A ~ P,A < P,0.000000
5,,,,,,,,,,A < C,A < C,0.000000
6,ADHD symptoms,Hyperactive-impulsive,Teacher,M ~ P,M ~ P,0.665000,C ~ M,C > M,0.002600,A < M,A < M,0.000000
7,,,,,,,C ~ P,C > P,0.000600,A ~ P,A < P,0.000000
8,,,,,,,,,,A < C,A < C,0.000000
9,ADHD symptoms,Hyperactive-impulsive,Parent,M > P,M ~ P,0.665000,C ~ M,C > M,0.002600,A < M,A < M,0.000000
